# First Baseline Model - Random Forest

#### ``Objectives``
1. Implement a Baseline Models for run value prediction PRE BATTED-BALL
2. Turn to a Random Forest for the another baseline model

### Import Libraries

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# decision tree
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestRegressor


# random forest|

# misc
import imblearn
import os
import glob


#### Clear output and stored data:



In [89]:
os.system('clear') 

0

---
#### <span style="color:chocolate">  Step 1: Data ingestion </span>

I already created the training data in another file:
 <span style="color:gray">TrackMan data of 2024 spring season</span> function below according to the following guidelines:

 a) Read all the csv files in the directory and merge them into a single dataframe \
 b) Save the dataframe to a csv file

In [90]:
# dont need to run this again since already created the training data

def load_data(path: str, num_columns=60) -> pd.DataFrame:
    """
    Loads and merges CSV files from the specified directory, excluding files with 'player positioning' in their names.
    
    Parameters:
    path (str): The directory path containing the CSV files.

    Returns:
    pandas.DataFrame: The merged DataFrame containing data from the selected CSV files.
    """
    try:
        # Ensure the directory exists
        if not os.path.exists(path):
            raise FileNotFoundError(f"The directory '{path}' does not exist.")

        # Get all files in the directory that end with .csv, excluding those with 'player positioning' in the name
        all_files = [
            file for file in glob.glob(f"{path}/*.csv") if 'player positioning' not in file
        ]

        # Raise an exception if no valid files are found
        if not all_files:
            raise ValueError(f"No valid CSV files found in the directory '{path}'.")

        # Set the indices of the columns to keep
        columns_to_keep = list(range(num_columns))  # will set that in the function call but usually 60 will be fine

        # Read and merge the filtered files with the specified columns
        df_list = [pd.read_csv(filename, usecols=columns_to_keep) for filename in all_files]
        merged_df = pd.concat(df_list, ignore_index=True)

        # Save the merged DataFrame to a CSV
        output_path = "/Users/tommayer/Desktop/games_test.csv"
        merged_df.to_csv(output_path, index=False)

        return merged_df

    except FileNotFoundError as fnf_error:
        print(f"Error: {fnf_error}")
    except ValueError as val_error:
        print(f"Error: {val_error}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


#### NOTE: 
I don't know if it's smart to load the data and concatenate all rows every time.  I could make it more like appending rows to the dataframe.

In [91]:
# ONLY COLUMNS with data from pre-batted ball + RunsScored as our target variable
required_columns = ['TaggedPitchType', 'AutoPitchType', 'RelSpeed', 'RelHeight', 'VertRelAngle', 'HorzRelAngle', 'PitchCall',
                      'SpinRate', 'SpinAxis', 'Tilt', 'Extension','InducedVertBreak', 'HorzBreak', 'VertApprAngle', 'HorzApprAngle']

# RunsScored column is gone

In [92]:
path = "/Users/tommayer/Desktop/training_data.csv"
#data = load_data(path)
data = pd.read_csv(path, usecols=required_columns)
## drastically reduces the number of rows and columns -> way less memory

Make new column for Whiffs


In [93]:
# Create boolean column for Whiffs (True if PitchCall is 'StrikeSwinging', False otherwise)
data['Whiff'] = data['PitchCall'] == 'StrikeSwinging'

In [94]:
data.drop(columns=['PitchCall'], inplace=True)

In [95]:
# peer at data and get a sense of the shape
data.head(5)
#print(f'Data shape: {data.shape}')

,TaggedPitchType,AutoPitchType,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,Extension,InducedVertBreak,HorzBreak,VertApprAngle,HorzApprAngle,Whiff
0,Slider,Slider,86.34831,-5.087035,-0.556059,2514.190308,69.694698,8:15,6.89596,5.20061,-1.26150,-6.71201,-12.231122,-1.751516,False
1,Fastball,Four-Seam,94.49974,-3.133086,-0.492520,2095.787589,190.374426,12:15,6.87165,5.83655,20.20828,3.49654,-5.365324,0.134391,False
2,Fastball,Four-Seam,94.81021,-3.910073,-1.135525,1996.806823,178.803234,12:00,6.94572,5.67326,22.06875,-0.43740,-5.815582,-1.213668,False
3,Slider,Slider,86.30865,-1.385858,-0.791508,3480.483920,100.930240,9:15,6.96039,5.43599,2.34610,-6.38485,-7.862841,-1.929632,False
4,Slider,Slider,87.45870,-4.605749,-1.323250,1287.761851,79.042003,8:45,6.99938,5.28786,0.27646,-4.37162,-11.264591,-2.102031,False


---
#### <span style="color:chocolate"> Step 2: Exploratory data analysis (EDA) </span>
- check for missing values
- check for duplicates
- check for outliers
- check for class imbalance


Rows to be dropped if N/A: 
- our target variables
- name, date, location, team??


In [96]:
new_required_columns = ['TaggedPitchType', 'AutoPitchType', 'RelSpeed', 'RelHeight', 'VertRelAngle', 'HorzRelAngle',
                      'SpinRate', 'SpinAxis', 'Tilt', 'Extension','InducedVertBreak', 'HorzBreak', 'VertApprAngle', 'HorzApprAngle']


In [97]:
# drop rows without certain columns
data = data.dropna(subset=new_required_columns)

In [98]:
# check how many rows were dropped
print(f'Number of rows dropped: {data.shape[0] - len(data)}')


Number of rows dropped: 0


Check data types:

In [99]:
print(data.dtypes)

TaggedPitchType      object
AutoPitchType        object
RelSpeed            float64
VertRelAngle        float64
HorzRelAngle        float64
SpinRate            float64
SpinAxis            float64
Tilt                 object
RelHeight           float64
Extension           float64
InducedVertBreak    float64
HorzBreak           float64
VertApprAngle       float64
HorzApprAngle       float64
Whiff                  bool
dtype: object


In [100]:
independent_vars = data.drop(['Whiff'], axis=1)
dependent_var = data['Whiff']

In [101]:
ind_names = independent_vars.columns
# reference this down below so i dont have to list them out there
# Get column names by data type
categorical_columns = independent_vars.select_dtypes(include=['object']).columns.tolist()
numerical_columns = independent_vars.select_dtypes(include=['float64', 'int64']).columns.tolist()



In [102]:
print(f'Categorical columns: {categorical_columns}')
print(f'Numerical columns: {numerical_columns}')

Categorical columns: ['TaggedPitchType', 'AutoPitchType', 'Tilt']
Numerical columns: ['RelSpeed', 'VertRelAngle', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 'RelHeight', 'Extension', 'InducedVertBreak', 'HorzBreak', 'VertApprAngle', 'HorzApprAngle']


### Feature Engineering:

In [103]:
# Add interaction terms between key features
data['SpeedSpin'] = data['RelSpeed'] * data['SpinRate']
data['BreakComposite'] = data['InducedVertBreak'] * data['HorzBreak']

# Add polynomial features for important numerical variables
data['RelSpeed_Squared'] = data['RelSpeed'] ** 2
data['SpinRate_Squared'] = data['SpinRate'] ** 2 

In [104]:
# Create interaction terms between top features
data['VertAngle_Combined'] = data['VertRelAngle'] * data['VertApprAngle']
data['HorzAngle_Combined'] = data['HorzRelAngle'] * data['HorzApprAngle']

# Create velocity-based features
data['SpeedSpin_Ratio'] = data['RelSpeed'] / data['SpinRate']
data['PowerFeature'] = data['RelSpeed'] * data['RelSpeed']  # Squared velocity

# Create break efficiency features
data['TotalBreak'] = np.sqrt(data['InducedVertBreak']**2 + data['HorzBreak']**2)
data['BreakRatio'] = data['InducedVertBreak'] / data['HorzBreak']

---
#### <span style="color:chocolate"> Step 3: Data Preprocessing </span>
- drop columns that are not useful?
- encode labels 
- split into training and testing data
- standardize data

Working with certain data types: \
a) numerical data (float, int)  \
    - scale data \
    - RelSpeed, SpinRate, InducedVertBreak, HorzBreak, ExitSpeed, etc \
    \
b) categorical data (object/string) \
    - encode data (one-hot encoding with sklearn LabelEncoder) \
    - TaggedPitchType, AutoPitchType, PitchCall, KorBB, TaggedHitType, PlayResult


In [105]:
def preprocess_data(data: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.Series, pd.Series, pd.Series]:
    """
    Preprocesses the data by identifying column types, encoding categorical data, and scaling numerical data.
    Returns train/test/validation splits of features and target.

    A series is a 1D array-like or list-like object that contains a single column of data (test and validation sets).
    """
    
    # 1. Identify column types
    numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = data.select_dtypes(include=['object']).columns

    # 2. Handle categorical data
    # For simple categorical variables, use Label Encoding
    """ for col in ['TaggedPitchType', 'AutoPitchType']:
        le = LabelEncoder()
        data[f'{col}_encoded'] = le.fit_transform(data[col])
        ## Note: label encoding assumes an order to the categories """

    # For nominal variables with many categories, use One-Hot Encoding
    data = pd.get_dummies(data, columns=categorical_columns)

    # 3. Scale numerical features
    scaler = StandardScaler()
    numerical_features = numerical_columns
    data[numerical_features] = scaler.fit_transform(data[numerical_features])

    # 4. Split into features and target
    X = data.drop(['Whiff'], axis=1)
    y = data['Whiff']

    # 6. Split into train/test/validation sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    

    print(f'X_train shape: {X_train.shape}')
    print(f'X_val shape: {X_val.shape}')
    print(f'X_test shape: {X_test.shape}')
    print(f'y_train shape: {y_train.shape}')
    print(f'y_val shape: {y_val.shape}')
    print(f'y_test shape: {y_test.shape}')

    return X_train, X_test, X_val, y_train, y_test, y_val

In [106]:
X_train, X_test, X_val, y_train, y_test, y_val = preprocess_data(data)

X_train shape: (673708, 91)
X_val shape: (168428, 91)
X_test shape: (210534, 91)
y_train shape: (673708,)
y_val shape: (168428,)
y_test shape: (210534,)


In [113]:
# Before training the model, clean the data by replacing infinities and checking for very large values
# Add this after your feature engineering steps and before model training

# Replace infinities with NaN
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_val = X_val.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Fill NaN values with the median of each column
for col in X_train.columns:
    median_val = X_train[col].median()
    X_train[col] = X_train[col].fillna(median_val)
    X_val[col] = X_val[col].fillna(median_val)
    X_test[col] = X_test[col].fillna(median_val)

# Clip very large values to a reasonable range
# You might need to adjust these values based on your data
max_val = 1e6
min_val = -1e6
X_train = X_train.clip(min_val, max_val)
X_val = X_val.clip(min_val, max_val)
X_test = X_test.clip(min_val, max_val)



This next part helps with class imbalance - assign weights to the classes based on their frequency

In [111]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.class_weight import compute_sample_weight

# Compute sample weights based on class frequencies
sample_weights = compute_sample_weight('balanced', y_train)

---
#### <span style="color:chocolate"> Step 4: Modeling </span>
- train a decision tree
- train a random forest
- train a gradient boosting machine (XGBoost)
- compare the three models

#### Decision Tree:
- enseble method - combines multiple decision trees to make a prediction
- uses bootstrap aggregating (bagging) - trains each tree on a different bootstrap sample of the data
- uses random subspace method - trains each tree on a different random subset of the features
- reduces variance and avoids overfitting
- can handle both numerical and categorical data
- easy to understand and interpret
- prone to overfitting if not tuned properly


In [114]:
# Import the correct model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Create and train the model
model_rf = RandomForestRegressor(
    n_estimators=100,      # Number of trees
    max_depth=20,        # Maximum depth of trees (None means unlimited)
    min_samples_split=2,   # Minimum samples required to split
    random_state=1,       # For reproducibility
    n_jobs=-1        # Use all CPU cores
)
model_rf.fit(X_train, y_train, sample_weight=sample_weights)

# Make predictions
predictions_rf = model_rf.predict(X_val)

# Evaluate the model (using regression metrics instead of accuracy)
mse = mean_squared_error(y_val, predictions_rf)
rmse = mean_squared_error(y_val, predictions_rf, squared=False)  # Root Mean Squared Error
r2 = r2_score(y_val, predictions_rf)

print(f'Random Forest Performance:')
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'R² Score: {r2:.4f}')

# Feature importance (optional)
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model_rf.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

Random Forest Performance:
Mean Squared Error: 0.1103
Root Mean Squared Error: 0.3321
R² Score: -0.2185

Top 10 Most Important Features:
               feature  importance
9        VertApprAngle    0.124384
5            RelHeight    0.095058
1         VertRelAngle    0.088928
16  HorzAngle_Combined    0.061034
15  VertAngle_Combined    0.057973
6            Extension    0.057007
2         HorzRelAngle    0.056116
10       HorzApprAngle    0.044439
19          TotalBreak    0.040620
17     SpeedSpin_Ratio    0.039886


/Users/tommayer/.pyenv/versions/3.12.6/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Hyper-Parameter Tuning:
- n_estimators
- max_depth
- min_samples_split
- min_samples_leaf
- max_features


In [88]:
from sklearn.model_selection import RandomizedSearchCV

# Define a smaller parameter grid
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Use RandomizedSearchCV instead of GridSearchCV
random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=10,  # Number of parameter settings sampled
    cv=3,  # Reduce the number of cross-validation folds
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

random_search.fit(X_train, y_train)
print(f"Best parameters: {random_search.best_params_}")

Best parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}


Best parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}

Cross-Validation:
- use cross-validation to evaluate the model's performance on the training data
- use the validation set to tune the hyperparameters
- use the test set to evaluate the final model's performance

In [115]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    model_rf, 
    X_train, 
    y_train, 
    cv=5, 
    scoring='neg_mean_squared_error'
)
rmse_scores = np.sqrt(-cv_scores)
print(f"Cross-validation RMSE: {rmse_scores.mean():.4f} (+/- {rmse_scores.std() * 2:.4f})")

Cross-validation RMSE: 0.2674 (+/- 0.0004)


Cross-validation RMSE: 0.2674 (+/- 0.0004)



## Gradient Boosting Machines

In [116]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Create and train the model
model_xgb = XGBRegressor(
    n_estimators=100,
    learning_rate=0.2,
    max_depth=9,
    random_state=42
) # define the model

model_xgb.fit(X_train, y_train) # train the model

# Make predictions
predictions_xgb = model_xgb.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, predictions_xgb)
rmse = mean_squared_error(y_val, predictions_xgb, squared=False)
r2 = r2_score(y_val, predictions_xgb)

print(f'XGBoost Performance:')

print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'R² Score: {r2:.4f}')

XGBoost Performance:
Mean Squared Error: 0.0845
Root Mean Squared Error: 0.2906
R² Score: 0.0667


/Users/tommayer/.pyenv/versions/3.12.6/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [117]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Define the parameter space
param_distributions = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 7)
}

# Create RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

# Fit the random search
random_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters found:")
for param, value in random_search.best_params_.items():
    print(f"{param}: {value}")
print(f"\nBest RMSE: {(-random_search.best_score_)**0.5:.4f}")

# Create model with best parameters
best_model_xgb = XGBRegressor(**random_search.best_params_, random_state=42)
best_model_xgb.fit(X_train, y_train)

# Evaluate best model
best_predictions = best_model_xgb.predict(X_val)
best_rmse = mean_squared_error(y_val, best_predictions, squared=False)
best_r2 = r2_score(y_val, best_predictions)

print(f'\nBest Model Performance:')
print(f'Root Mean Squared Error: {best_rmse:.4f}')
print(f'R² Score: {best_r2:.4f}')


/Users/tommayer/.pyenv/versions/3.12.6/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters found:
colsample_bytree: 0.815750979360025
learning_rate: 0.21518913081944233
max_depth: 9
min_child_weight: 2
n_estimators: 290
subsample: 0.9468795734220015

Best RMSE: 0.2720

Best Model Performance:
Root Mean Squared Error: 0.2677
R² Score: 0.2080


/Users/tommayer/.pyenv/versions/3.12.6/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Best parameters found:
colsample_bytree: 0.815750979360025
learning_rate: 0.21518913081944233
max_depth: 9
min_child_weight: 2
n_estimators: 290
subsample: 0.9468795734220015

Best RMSE: 0.2720

Best Model Performance:
Root Mean Squared Error: 0.2677
R² Score: 0.2080

XGBoost Performance:
Mean Squared Error: 0.0845
Root Mean Squared Error: 0.2906
R² Score: 0.0667

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, roc_auc_score

# Create stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Custom scorer that weights recall more heavily
scorer = make_scorer(roc_auc_score, needs_proba=True)

# Perform cross-validation
cv_scores = cross_val_score(
    model_rf,
    X_train,
    y_train,
    cv=skf,
    scoring=scorer,
    n_jobs=-1
)

print(f"CV ROC-AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Run Expectancy Model from Cursor

In [ ]:
def prepare_run_expectancy_data(data):
    """
    Prepare run expectancy data with proper game/inning boundaries
    """
    # Assuming you have or can add these columns:
    required_columns = [
        'GameID',        # Unique identifier for each game
        'InningID',      # Inning number
        'InningHalf',    # Top/Bottom
        'RunsScored'
    ]
    
    # Sort data chronologically
    data = data.sort_values(['GameID', 'InningID', 'InningHalf'])
    
    # Calculate runs scored for rest of inning
    def calculate_future_runs(group):
        # Sum runs scored after each pitch until end of inning
        group['future_runs'] = group['RunsScored'].iloc[::-1].cumsum().iloc[::-1] - group['RunsScored']
        return group
    
    # Group by game and inning to respect boundaries
    data = data.groupby(['GameID', 'InningID', 'InningHalf']).apply(calculate_future_runs)
    
    return data

def train_run_expectancy_model(data):
    """
    Train model to predict runs scored in remainder of inning
    """
    # Features that could predict run expectancy
    features = [
        'RelSpeed', 'RelHeight', 'VertRelAngle', 'HorzRelAngle',
        'SpinRate', 'SpinAxis', 'InducedVertBreak', 'HorzBreak',
        'VertApprAngle', 'HorzApprAngle',
        # Add dummy variables for pitch types
        *[col for col in data.columns if col.startswith('TaggedPitchType_')]
    ]
    
    X = data[features]
    y = data['future_runs']  # Target is runs scored in remainder of inning
    
    # Split respecting game boundaries
    game_ids = data['GameID'].unique()
    train_games, test_games = train_test_split(game_ids, test_size=0.2, random_state=42)
    
    X_train = X[data['GameID'].isin(train_games)]
    X_test = X[data['GameID'].isin(test_games)]
    y_train = y[data['GameID'].isin(train_games)]
    y_test = y[data['GameID'].isin(test_games)]
    
    model = GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    return model, X_test, y_test

def calculate_pitch_run_values(data, model):
    """
    Calculate run value for each pitch
    """
    # Group by game and inning
    def process_inning(group):
        # Get features for prediction
        features = [
            'RelSpeed', 'RelHeight', 'VertRelAngle', 'HorzRelAngle',
            'SpinRate', 'SpinAxis', 'InducedVertBreak', 'HorzBreak',
            'VertApprAngle', 'HorzApprAngle',
            *[col for col in group.columns if col.startswith('TaggedPitchType_')]
        ]
        
        # Calculate run expectancy before and after each pitch
        re_before = model.predict(group[features])
        re_after = np.roll(re_before, -1)  # Shift predictions up by one
        re_after[-1] = 0  # Last pitch in inning has 0 future run expectancy
        
        # Calculate run value
        run_value = re_before - re_after + group['RunsScored']
        
        return run_value
    
    data['run_value'] = data.groupby(['GameID', 'InningID', 'InningHalf']).apply(process_inning)
    
    return data

# Usage example:
def analyze_run_values():
    # Prepare data
    data = prepare_run_expectancy_data(data)
    
    # Train model
    model, X_test, y_test = train_run_expectancy_model(data)
    
    # Calculate run values
    data = calculate_pitch_run_values(data, model)
    
    # Analyze results
    print("\nAverage Run Values by Pitch Type:")
    print(data.groupby('TaggedPitchType')['run_value'].mean().sort_values(ascending=False))
    
    # Visualize
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='TaggedPitchType', y='run_value', data=data)
    plt.title('Run Values by Pitch Type')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()